In [1]:
from IPython.core.debugger import prompt
from openai import OpenAI
import os
import json

# Set your API key
api_key = ""

# Initialize the OpenAI client with your API key
#OpenAI.api_key = api_key
client = OpenAI(api_key = api_key)

In [15]:
import json

def read_json_file(filename):
    data = []
    with open(filename) as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}. Error: {e}")
    return data


gamename = 'night'

datapath = f'data/Walkthrough Text/walkthrough_{gamename}_obs&inv_M.json'
inputData = read_json_file(datapath)


In [16]:
print(len(inputData))
print(inputData[0]['observation'])

56

Hall Outside Computer Site
This is the north end of the third floor of Hodges Hall.  To the southwest is the entrance to the Computer Site, and to the east are stairs up to the fourth floor.  Stairs down to lower floors are to the west.

To one side, a water fountain hums quietly.


In [4]:
categorize_system_prompt = r"""You are an AI designed to be good at entity analysis and relation extraction."""

In [5]:
%%markdown
prompt = r"""
You're going to extract the triples vector <sub, relation, obj> from an input Observation and Inventory, which the sentence comes from the text-based game Zork1. That means you only need to pay attention to where the character ('You') is located, what objects are in this location, and the properties of objects. 

Rules: 
(1) When representing the character's ('You') location, always use 'in' as the relation. If the input uses other forms of spatial description (like 'are in', 'are facing', 'are standing', 'are behind', 'are above', 'are below', or 'are in front'), these should be turned to 'in'. The location of 'You' should be represented only once using the 'in' relationship. Also, if the start of input seems to be a location, use it as the location name, or if there is a more detailed description, such as '()', use the location with the details. 
Like: ""
Hall
You're in the middle of a long north/south hall. ..." use "the middle of a long north/south hall" rather than "Hall".
(2) For the relation between locations and objects, use 'have' to represent the object in the location. The relations should only add the object itself in triples vector as obj; for the decorative descriptions of the object, please use the rule of Tips 3. Like "You are standing in an open field west of a white house, with a boarded front door. There is a small mailbox here." gives "<west of a white house, have, mailbox>".
(3) When representing an object with additional decorative descriptions, such as 'on', 'in', 'with', 'verb+ing', etc., use their preposition or a simple representation (Like "a buzzing water fountain has been moved" gives "<water fountain, is, buzzing>, <water fountain, is, moved>") as the relationships (Use with Rule 5). But if an object has another, use 'have'. Like "One particularly large tree with some low branches stands at the edge of the path" gives "<large tree, with, low branches>, <large tree, at, the edge of the path>" 
(4) If the input asks for something to 'continue xxx' or 'do xxx', use '<location/door/objects/etc., need/require/etc., something to xxx>'. Like "Forest This is a forest, with trees in all directions. You would need a machete to go further west." gives "<Forest, need, machete to west>".
(5) If an object's properties are nearly close, you can summarize them in one property as the 'obj' represented in the triples vector.
(6) If nothing happened, output none.
(7) If the Inventory is not empty, please create relationships with the objects and character ('You'). If Inventory is empty or shows 'You don't have anything', output none.
(8) For any object in the current location leading by a direction (like 'to the north', 'up to', 'down' and so on) to a new location or any direction can go from observation, use the <Current Location, direction, new location/none> to represent. Like "Hall Outside Computer Site This is the north end of the third floor of Hodges Hall.  To the southwest is the entrance to the Computer Site" can be represented as "<Hall Outside Computer Site, southwest, Computer Site>".


Now, please extract the relationships for the input step by step and merge the results as one in "|start| * |end|", where * are the results.


"""

prompt = r"""
You're going to extract the triples vector <sub, relation, obj> from an input Observation and Inventory, which the sentence comes from the text-based game Zork1. That means you only need to pay attention to where the character ('You') is located, what objects are in this location, and the properties of objects. 

Rules: 
(1) When representing the character's ('You') location, always use 'in' as the relation. If the input uses other forms of spatial description (like 'are in', 'are facing', 'are standing', 'are behind', 'are above', 'are below', or 'are in front'), these should be turned to 'in'. The location of 'You' should be represented only once using the 'in' relationship. Also, if the start of input seems to be a location, use it as the location name, or if there is a more detailed description, such as '()', use the location with the details. 
Like: ""
Hall
You're in the middle of a long north/south hall. ..." use "the middle of a long north/south hall" rather than "Hall".
(2) For the relation between locations and objects, use 'have' to represent the object in the location. The relations should only add the object itself in triples vector as obj; for the decorative descriptions of the object, please use the rule of Tips 3. Like "You are standing in an open field west of a white house, with a boarded front door. There is a small mailbox here." gives "<west of a white house, have, mailbox>".
(3) When representing an object with additional decorative descriptions, such as 'on', 'in', 'with', 'verb+ing', etc., use their preposition or a simple representation (Like "a buzzing water fountain has been moved" gives "<water fountain, is, buzzing>, <water fountain, is, moved>") as the relationships (Use with Rule 5). But if an object has another, use 'have'. Like "One particularly large tree with some low branches stands at the edge of the path" gives "<large tree, with, low branches>, <large tree, at, the edge of the path>" 
(4) If the input asks for something to 'continue xxx' or 'do xxx', use '<location/door/objects/etc., need/require/etc., something to xxx>'. Like "Forest This is a forest, with trees in all directions. You would need a machete to go further west." gives "<Forest, need, machete to west>".
(5) If an object's properties are nearly close, you can summarize them in one property as the 'obj' represented in the triples vector.
(6) If nothing happened, output none.
(7) If the Inventory is not empty, please create relationships with the objects and character ('You'). If Inventory is empty or shows 'You don't have anything', output none.
(8) For any object in the current location leading by a direction (like 'to the north', 'up to', 'down' and so on) to a new location or any direction can go from observation, use the <Current Location, direction, new location/none> to represent. Like "Hall Outside Computer Site This is the north end of the third floor of Hodges Hall.  To the southwest is the entrance to the Computer Site" can be represented as "<Hall Outside Computer Site, southwest, Computer Site>".


Now, please extract the relationships for the input step by step and merge the results as one in "|start| * |end|", where * are the results.


"""


In [6]:
# v11
prompt = r"""
You're going to extract the triples vector <sub, relation, obj> from an input Observation and Inventory, which the sentence comes from the text-based game Zork1. That means you only need to pay attention to where the character ('You') is located, what objects are in this location, and the properties of objects. 

Rules: 
(1) Always use 'in' as the relation when representing the character's location. If the input uses other forms of spatial description (like 'are in', 'are facing', 'are standing', 'are behind', 'are above', 'are below', 'are in front of'), convert them to the 'in' relationship. Represent the character's location only once. If the input begins with a Room name, use it as the location (Use with Rule 2). For example: "Stairwell (First Floor) \n You're in the north stairwell." should be represented as "<You, in, Stairwell(First Floor)>" or "<You, in, north stairwell(First Floor)>".
(2) If a more detailed description is provided in parentheses (if '()' followed Room, include '()'), include that in the location only when you think the Room's name is not unique enough. So please check it all the time when using Rule 1. For example: "Hall. You're in the middle of a long north-south hall." should be represented as "<You, in, the middle of a long north-south hall>" because 'Hall' is not unique.
(3) Use 'have' as the relation to represent objects present in the location. Focus on the object itself as the 'obj' in the triple. Ignore decorative details unless specified otherwise. For example: "You are standing in an open field west of a white house. There is a small mailbox here." should be represented as "<west of a white house, have, mailbox>."
(4) When objects are described with additional details (e.g., 'on', 'in', 'with', or verbs like 'buzzing'), use the preposition or a simple 'is' to represent the object’s state (Use with Rule 5). If an object contains or includes another, use 'have' to represent this. For example: "A buzzing water fountain has been moved." should be represented as "<water fountain, is, buzzing>, <water fountain, is, moved>." Another example: "One particularly large tree with some low branches stands at the edge of the path." should be represented as "<large tree, with, low branches>, <large tree, at, edge of path>."
(5) If the input specifies a requirement or action needed to continue, use "<location/door/object, need/require, something to action>." For example: "Forest. You would need a machete to go further west." should be represented as "<Forest, need, machete to go west>."
(6) If an object's properties are nearly close, you can summarize them in one property as the 'obj' represented in the triples vector.
(7) If nothing happened, output none.
(8) If the Inventory is not empty, please create relationships with the objects and character ('You'). If Inventory is empty or shows 'You don't have anything', output none.
(9) For objects or locations mentioned in conjunction with a direction (e.g., 'to the north', 'up to', 'down'), use "<current location, direction, new location/to direction>." For example: "Hall. To the southwest is the entrance to the Computer Site, and you can go east here as well as go up with a stair." should be represented as "<Hall, southwest, Computer Site>, <Hall, east, to east>, <Hall, up, to up>."


Now, please extract the relationships for the input step by step and merge all the results into one "|start| * |end|", where * are the results.

"""

In [7]:
def GetInputFormat(state):
    return f"""Input:\n{{\nObservation: "{state['observation']}"\nInventory: "{state['inventory']}"\n}}"""
    

In [8]:
tasks = []
i = 0
for t in inputData:
    description = prompt + GetInputFormat(t) + "\nPlease follow the rules, think, and do it step by step."
    task = {
        "custom_id": f"task-{i}",
        "url": "/v1/chat/completions",
        'method': 'POST',
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4o",
            "temperature": 0.2,
            "messages": [
                {
                    "role": "system", 
                    "content": categorize_system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }
    i += 1
    tasks.append(task)

print(len(tasks))

109


In [9]:
print(tasks[1]['body']['messages'][1]['content'])


You're going to extract the triples vector <sub, relation, obj> from an input Observation and Inventory, which the sentence comes from the text-based game Zork1. That means you only need to pay attention to where the character ('You') is located, what objects are in this location, and the properties of objects. 

Rules: 
(1) Always use 'in' as the relation when representing the character's location. If the input uses other forms of spatial description (like 'are in', 'are facing', 'are standing', 'are behind', 'are above', 'are below', 'are in front of'), convert them to the 'in' relationship. Represent the character's location only once. If the input begins with a Room name, use it as the location (Use with Rule 2). For example: "Stairwell (First Floor) \n You're in the north stairwell." should be represented as "<You, in, Stairwell(First Floor)>" or "<You, in, north stairwell(First Floor)>".
(2) If a more detailed description is provided in parentheses (if '()' followed Room, includ

In [10]:
##%%markdown
file_name = f"data/ChatGPT/4o/4o_RE_inputs_{gamename}_zero-cot_v11_t0.2"

with open(file_name+'_part1.jsonl', 'w') as file:
    for obj in tasks[:int(len(tasks)/2)]:
        file.write(json.dumps(obj) + '\n')
    print(f"Results saved in {file_name}_part1.jsonl")

with open(file_name+'_part2.jsonl', 'w') as file:
    for obj in tasks[int(len(tasks)/2):]:
        file.write(json.dumps(obj) + '\n')
    print(f"Results saved in {file_name}_part2.jsonl")

Results saved in data/ChatGPT/4o/4o_RE_inputs_karn_zero-cot_v11_t0.2_part1.jsonl
Results saved in data/ChatGPT/4o/4o_RE_inputs_karn_zero-cot_v11_t0.2_part2.jsonl


In [11]:
%%markdown
file_name = f"data/ChatGPT/4o/4o_RE_inputs_{gamename}_zero-cot_v11_t0.2.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')
    print(f"Results saved in {file_name}")

file_name = f"data/ChatGPT/4o/4o_RE_inputs_{gamename}_zero-cot_v11_t0.2.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')
    print(f"Results saved in {file_name}")


In [12]:
for data in inputData:
    print(data['observation'])
    print('='*60)





A Galactic servant, that's what you feel like at the moment.  You'd think that the Timelords would be pleased that you were not only responsible for discovering that Morbius' brain had been secretly kept alive, but that you were instrumental in destroying the revived villian.  Do they offer you their gratitude?  No, they merely point out that you left the Brain of Morbius on Karn and demand that you retrieve it and bring it back to Gallifrey.  That's gratitude for you.

However, as you value your own freedom, you really don't have much choice but to do as the Timelords request.  At least they gave you the coordinates for Karn, but you did notice that they refused to give you return coordinates for Gallifrey, saying you'd receive them once you'd procured the brain.  So much for Gallifreyan hospitality...

Your thoughts are interrupted as you hear a familiar thud.  The TARDIS has landed on Karn.

Return to Karn
by Patrick Wigfull
Based on the television programme "Doctor Who"
  Type 

In [217]:
output_filename = 'data/ChatGPT/4o/4o_RE_outputs_night_zero-cot_v11_t0.4_top10.jsonl'
output = read_json_file(output_filename)

for o in output:
    print(o['response']['body']['choices'][0]['message']['content'])
    print('='*60)

To extract the triples from the given input, let's break down the information step by step according to the rules:

1. **Character's Location:**
   - The input begins with "Hall Outside Computer Site," which indicates the location. According to Rule 1, we represent the character's location as "<You, in, Hall Outside Computer Site>."

2. **Objects in the Location:**
   - The text mentions "a water fountain hums quietly." According to Rule 4, we represent this as "<water fountain, is, humming>."

3. **Directions and Connections:**
   - The text states "To the southwest is the entrance to the Computer Site." According to Rule 9, we represent this as "<Hall Outside Computer Site, southwest, Computer Site>."
   - It also mentions "to the east are stairs up to the fourth floor." We represent this as "<Hall Outside Computer Site, east, stairs up to fourth floor>."
   - Additionally, "Stairs down to lower floors are to the west." We represent this as "<Hall Outside Computer Site, west, stairs 

In [218]:
output_filename = 'data/ChatGPT/4o/4o_RE_outputs_night_zero-cot_v11_t0.6_top10.jsonl'
output = read_json_file(output_filename)

for o in output:
    print(o['response']['body']['choices'][0]['message']['content'])
    print('='*60)

Let's analyze the input step by step according to the provided rules:

1. **Location Extraction**:  
   The input begins with "Hall Outside Computer Site," which will be used as the location for the character. According to Rule 1, we represent the character's location as:  
   - `<You, in, Hall Outside Computer Site>`

2. **Object and Property Extraction**:  
   The sentence describes a water fountain that "hums quietly." According to Rule 4, we represent this object's property as:  
   - `<water fountain, is, humming>`

3. **Directional and Connection Relationships**:  
   The text describes directions and connections to other locations:  
   - "To the southwest is the entrance to the Computer Site." According to Rule 9, this is represented as:  
     - `<Hall Outside Computer Site, southwest, Computer Site>`
   - "To the east are stairs up to the fourth floor." This indicates a direction upwards:  
     - `<Hall Outside Computer Site, east, stairs to fourth floor>`
   - "Stairs down 

In [226]:
output_filename = 'data/ChatGPT/4o/4o_RE_outputs_night_zero-cot_v11_t0.2_top10.jsonl'
output = read_json_file(output_filename)

for o in output:
    print(o['response']['body']['choices'][0]['message']['content'])
    print('='*60)

Let's extract the triples step by step according to the rules provided:

1. **Character's Location:**
   - The input begins with "Hall Outside Computer Site." According to Rule 1, we use this as the location.
   - Triple: `<You, in, Hall Outside Computer Site>`

2. **Objects in the Location:**
   - The description mentions "a water fountain hums quietly."
   - According to Rule 4, we represent the object's state.
   - Triple: `<water fountain, is, humming>`

3. **Directions and Connections:**
   - "To the southwest is the entrance to the Computer Site."
   - According to Rule 9, we represent the direction and connection.
   - Triple: `<Hall Outside Computer Site, southwest, Computer Site>`
   
   - "To the east are stairs up to the fourth floor."
   - According to Rule 9, we represent the direction and connection.
   - Triple: `<Hall Outside Computer Site, east, to east>`

   - "Stairs down to lower floors are to the west."
   - According to Rule 9, we represent the direction and conne

In [13]:
output_filename = 'data/ChatGPT/4o/4o_RE_outputs_night_zero-cot_v11_t0.2.jsonl'
output = read_json_file(output_filename)

for o in output:
    print(o['response']['body']['choices'][0]['message']['content'])
    print('='*60)

To extract the triples vector from the given input, let's follow the rules step by step:

1. **Identify the Character's Location:**
   - The input begins with "Hall Outside Computer Site," which is the location. According to Rule 1 and Rule 2, we represent the character's location as "<You, in, Hall Outside Computer Site>."

2. **Identify Objects in the Location:**
   - The observation mentions "a water fountain hums quietly." According to Rule 4, we represent this as "<water fountain, is, humming>."

3. **Identify Directions and Connections:**
   - The observation states:
     - "To the southwest is the entrance to the Computer Site." According to Rule 9, this is represented as "<Hall Outside Computer Site, southwest, Computer Site>."
     - "To the east are stairs up to the fourth floor." According to Rule 9, this is represented as "<Hall Outside Computer Site, east, to east>."
     - "Stairs down to lower floors are to the west." According to Rule 9, this is represented as "<Hall Ou

In [17]:
print(len(inputData))
print(len(output))

56
56


In [18]:
import re
import csv

def take_results_from_output(text):
    all_sections = re.findall(r"\|start\|(.+?)\|end\|", text.replace('\n', '').strip())
    #print(text.replace('\n', '').strip())
    # Extracting the last section
    if all_sections:
        last_section = all_sections[-1]
    
        # Splitting into individual triples
        triples = re.findall(r"<(.*?)>", last_section)
    
        # Parsing each triple into (subject, relation, object)
        parsed_triples = [tuple(triple.split(", ")) for triple in triples]
    
        # Displaying the result
        return parsed_triples
    else:
        parsed_triples = []
        raise ValueError(f"No valid sections found on: \n[{text}]")
    
    return ""

    
    
input_string = r"""To extract the triples vector from the given input, let's break down the observation and inventory step by step according to the rules provided:

1. **Character Location**: 
   - The observation starts with "Hall Outside Computer Site," indicating the current location. 
   - According to Rule 1, we represent the character's location using 'in'.
   - Triple: `<You, in, Hall Outside Computer Site>`

2. **Objects in the Location**:
   - The observation mentions a "water fountain" that "hums quietly."
   - According to Rule 2, we represent objects in the location using 'have'.
   - Triple: `<Hall Outside Computer Site, have, water fountain>`

3. **Directional Information**:
   - The observation provides directions to other locations:
     - "To the southwest is the entrance to the Computer Site."
     - "To the east are stairs up to the fourth floor."
     - "Stairs down to lower floors are to the west."
   - According to Rule 8, we represent these directions as relationships.
   - Triples:
     - `<Hall Outside Computer Site, southwest, Computer Site>`
     - `<Hall Outside Computer Site, east, fourth floor>`
     - `<Hall Outside Computer Site, west, none>`

4. **Inventory**:
   - The inventory states, "You are carrying nothing."
   - According to Rule 7, if the inventory is empty, we output none for inventory-related triples.

Now, let's merge the results following the format "|start| * |end|":

```
|start| <You, in, Hall Outside Computer Site> * <Hall Outside Computer Site, have, water fountain> * <Hall Outside Computer Site, southwest, Computer Site> * <Hall Outside Computer Site, east, fourth floor> * <Hall Outside Computer Site, west, none> |end|
```"""

res = take_results_from_output(input_string)
print(res)

[('You', 'in', 'Hall Outside Computer Site'), ('Hall Outside Computer Site', 'have', 'water fountain'), ('Hall Outside Computer Site', 'southwest', 'Computer Site'), ('Hall Outside Computer Site', 'east', 'fourth floor'), ('Hall Outside Computer Site', 'west', 'none')]


In [25]:
eval_data = []

for i in range(len(inputData)):
#for i in range(1):
    obs = inputData[i]['observation']
    inv = inputData[i]['inventory']
    
    content = output[i]['response']['body']['choices'][0]['message']['content']
    extracted_relations = take_results_from_output(content)
    extracted_relations_str = ""
    for er in extracted_relations:
     extracted_relations_str += str(er) + "\n"
    temp = {}
    temp['obs'] = obs
    temp['inv'] = inv
    temp['tv'] = extracted_relations_str
    #print(temp)
    eval_data.append(temp)
            
# Specify the CSV file name
eval_csv = f"data/ChatGPT/eval/eval_output_{gamename}.csv"

eval_json = f"data/ChatGPT/eval/eval_output_{gamename}.json"

# Writing to CSV file
with open(eval_csv, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['obs', 'inv', 'tv'])

    # Write the header (field names)
    writer.writeheader()

    # Write each row (dictionary in eval_data)
    for data in eval_data:
        #data['tv'] = str(data['tv'])
        writer.writerow(data)

print(f"Data saved to {eval_csv}")

# Writing to JSON
with open(eval_json, mode='w') as file:
    json.dump(eval_data, file, indent=4)  # Using indent=4 for pretty printing

print(f"Data saved to {eval_json}")



Data saved to data/ChatGPT/eval/eval_output_night.csv
Data saved to data/ChatGPT/eval/eval_output_night.json


In [26]:
print(len(eval_data))

56


In [27]:
eval_csv = f"data/ChatGPT/eval/eval_output_{gamename}_for_eval.csv"

eval_data = []

for i in range(len(inputData)):
#for i in range(1):
    obs = inputData[i]['observation']
    inv = inputData[i]['inventory']
    
    content = output[i]['response']['body']['choices'][0]['message']['content']
    extracted_relations = take_results_from_output(content)
    for tv in extracted_relations:
        temp = {}
        temp['id'] = i    
        temp['obs'] = obs
        temp['inv'] = inv
        temp['tv'] = tv
    #print(temp)
        eval_data.append(temp)

with open(eval_csv, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['id', 'obs', 'inv', 'tv'])

    # Write the header (field names)
    writer.writeheader()

    # Write each row (dictionary in eval_data)
    for data in eval_data:
        #data['tv'] = str(data['tv'])
        writer.writerow(data)

print(f"Data saved to {eval_csv}")

Data saved to data/ChatGPT/eval/eval_output_night_for_eval.csv
